# Quality Control and Validation

This notebook demonstrates quality control and validation capabilities for XCT analysis. Learn how to:

- **Analyze dimensional accuracy** (CAD comparison, tolerance compliance)
- **Quantify uncertainty** (measurement uncertainty, confidence intervals)
- **Validate results** (DragonFly comparison, ground truth validation)
- **Perform quality control** (reproducibility, accuracy metrics)

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Assess dimensional accuracy against design specifications
2. Quantify measurement uncertainty
3. Validate analysis results against ground truth
4. Compare with DragonFly software
5. Generate quality control reports

## ⚠️ Prerequisites

- **Notebook 01**: Basic understanding of loading and segmenting volumes
- **Notebook 02**: Understanding of preprocessing and filtering
- **Notebook 03**: Understanding of morphological analysis
- **Required packages**: Same as previous notebooks
- **Segmented volume**: Binary segmented volume ready for analysis
- **Optional**: CAD model, design specifications, DragonFly results

## 📖 Usage

1. Run all cells to initialize the widgets
2. Load a segmented volume
3. Configure dimensional accuracy parameters (design specs, tolerances)
4. Analyze uncertainty
5. Validate against ground truth or DragonFly
6. Generate quality control reports


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
from typing import Dict, List, Optional, Tuple, Any

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Quality Control and Validation")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Quality Control and Validation
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load quality modules
try:
    from src.analyzer import XCTAnalyzer
    from src.quality.dimensional_accuracy import (
        compute_geometric_deviations, analyze_tolerance_compliance,
        comprehensive_dimensional_analysis
    )
    from src.quality.uncertainty_analysis import (
        measurement_uncertainty, confidence_intervals,
        comprehensive_uncertainty_analysis
    )
    from src.quality.validation import (
        compare_with_dragonfly, validate_against_ground_truth,
        compute_accuracy_metrics
    )
    from src.core.metrics import compute_all_metrics
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Quality Control Dashboard

Use the interactive widgets below to perform quality control and validation analysis.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    current_volume = None
    quality_results = {}
    
    # ============================================
    # Section 1: Data Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path to segmented volume',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'NumPy'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='100px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Dimensional Accuracy
    # ============================================
    
    design_dim_x = widgets.FloatText(value=10.0, description='Design X (mm):', style={'description_width': 'initial'})
    design_dim_y = widgets.FloatText(value=10.0, description='Design Y (mm):', style={'description_width': 'initial'})
    design_dim_z = widgets.FloatText(value=10.0, description='Design Z (mm):', style={'description_width': 'initial'})
    
    tolerance = widgets.FloatText(value=0.1, description='Tolerance (mm):', style={'description_width': 'initial'})
    
    analyze_dimensional_button = widgets.Button(
        description='📏 Analyze Dimensional Accuracy',
        button_style='info',
        layout=widgets.Layout(width='250px')
    )
    
    dimensional_results_display = widgets.HTML(
        value="<p><i>No dimensional analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    dimensional_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 3: Uncertainty Analysis
    # ============================================
    
    voxel_uncertainty_x = widgets.FloatText(value=0.001, description='Voxel Unc. X (mm):', style={'description_width': 'initial'})
    voxel_uncertainty_y = widgets.FloatText(value=0.001, description='Voxel Unc. Y (mm):', style={'description_width': 'initial'})
    voxel_uncertainty_z = widgets.FloatText(value=0.001, description='Voxel Unc. Z (mm):', style={'description_width': 'initial'})
    
    segmentation_uncertainty = widgets.FloatText(value=0.5, description='Seg. Uncertainty (voxels):', style={'description_width': 'initial'})
    
    confidence_level = widgets.FloatSlider(
        value=0.95,
        min=0.80,
        max=0.99,
        step=0.01,
        description='Confidence Level:',
        style={'description_width': 'initial'}
    )
    
    analyze_uncertainty_button = widgets.Button(
        description='📊 Analyze Uncertainty',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    uncertainty_results_display = widgets.HTML(
        value="<p><i>No uncertainty analysis</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    uncertainty_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Section 4: Validation
    # ============================================
    
    validation_method = widgets.Dropdown(
        options=['DragonFly Comparison', 'Ground Truth Validation'],
        value='DragonFly Comparison',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    dragonfly_file = widgets.Text(
        value='',
        placeholder='Path to DragonFly results (CSV/JSON)',
        description='DragonFly File:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    ground_truth_file = widgets.Text(
        value='',
        placeholder='Path to ground truth (CSV/JSON)',
        description='Ground Truth File:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    validation_tolerance = widgets.FloatText(value=0.01, description='Tolerance (%):', style={'description_width': 'initial'})
    
    run_validation_button = widgets.Button(
        description='✅ Run Validation',
        button_style='success',
        layout=widgets.Layout(width='200px')
    )
    
    validation_results_display = widgets.HTML(
        value="<p><i>No validation results</i></p>",
        layout=widgets.Layout(height='200px', overflow='auto')
    )
    
    validation_visualization = Output(layout=widgets.Layout(height='400px'))
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


## 4. Widget Callbacks and Functions


In [4]:
if WIDGETS_AVAILABLE:
    
    def load_volume_callback(button):
        """Load segmented volume"""
        global analyzer, current_volume
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        file_path_obj = Path(file_path)
        if not file_path_obj.exists():
            data_path = project_root / 'data' / file_path
            if data_path.exists():
                file_path_obj = data_path
            else:
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            analyzer.load_volume(str(file_path_obj), normalize=True)
            current_volume = analyzer.volume
            progress_bar.value = 80
            
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Voxel Size:</b> {voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_dimensional_callback(button):
        """Analyze dimensional accuracy"""
        global analyzer, current_volume, quality_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing dimensional accuracy...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            design_specs = {
                'dimensions': {
                    'x': float(design_dim_x.value),
                    'y': float(design_dim_y.value),
                    'z': float(design_dim_z.value)
                },
                'tolerance': float(tolerance.value)
            }
            
            # Compute geometric deviations
            deviation_result = compute_geometric_deviations(current_volume, design_specs, voxel_size)
            progress_bar.value = 60
            
            # Analyze tolerance compliance
            tolerances = {
                'default': float(tolerance.value),
                'x': float(tolerance.value),
                'y': float(tolerance.value),
                'z': float(tolerance.value)
            }
            compliance_result = analyze_tolerance_compliance(current_volume, tolerances, voxel_size, design_specs)
            progress_bar.value = 80
            
            quality_results['dimensional'] = {
                'deviations': deviation_result,
                'compliance': compliance_result
            }
            
            # Display results
            html = f"""
            <h4>📏 Dimensional Accuracy Results</h4>
            <p><b>Dimensional Accuracy:</b> {deviation_result.get('dimensional_accuracy', 0):.2f}%</p>
            <p><b>RMS Deviation:</b> {deviation_result.get('rms_deviation', 0):.4f} mm</p>
            <p><b>Max Deviation:</b> {deviation_result.get('max_deviation', 0):.4f} mm</p>
            <h5>Deviations by Axis:</h5>
            <ul>
            """
            for axis in ['x', 'y', 'z']:
                if axis in deviation_result.get('deviations', {}):
                    dev = deviation_result['deviations'][axis]
                    pct = deviation_result.get('deviation_percentages', {}).get(axis, 0)
                    within = deviation_result.get('within_tolerance', {}).get(axis, False)
                    html += f"<li><b>{axis.upper()}:</b> {dev:.4f} mm ({pct:.2f}%) {'✅' if within else '❌'}</li>"
            html += "</ul>"
            dimensional_results_display.value = html
            
            # Visualize
            with dimensional_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                
                # Deviation bar chart
                axes[0].bar(['X', 'Y', 'Z'], 
                          [deviation_result['deviations'].get('x', 0),
                           deviation_result['deviations'].get('y', 0),
                           deviation_result['deviations'].get('z', 0)],
                          alpha=0.7, color=['steelblue', 'coral', 'lightgreen'])
                axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
                axes[0].axhline(float(tolerance.value), color='red', linestyle='--', label=f'Tolerance: ±{tolerance.value} mm')
                axes[0].axhline(-float(tolerance.value), color='red', linestyle='--')
                axes[0].set_ylabel('Deviation (mm)', fontsize=11)
                axes[0].set_title('Dimensional Deviations', fontsize=12, fontweight='bold')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Actual vs Design comparison
                actual_dims = deviation_result.get('actual_dimensions', {})
                design_dims = deviation_result.get('design_dimensions', {})
                axes[1].bar(['X', 'Y', 'Z'], 
                          [design_dims.get('x', 0), design_dims.get('y', 0), design_dims.get('z', 0)],
                          alpha=0.5, label='Design', color='lightblue')
                axes[1].bar(['X', 'Y', 'Z'],
                          [actual_dims.get('x', 0), actual_dims.get('y', 0), actual_dims.get('z', 0)],
                          alpha=0.7, label='Actual', color='steelblue')
                axes[1].set_ylabel('Dimension (mm)', fontsize=11)
                axes[1].set_title('Design vs Actual Dimensions', fontsize=12, fontweight='bold')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Dimensional analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in dimensional analysis: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def analyze_uncertainty_callback(button):
        """Analyze measurement uncertainty"""
        global analyzer, current_volume, quality_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Analyzing uncertainty...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            voxel_size_uncertainty = (
                float(voxel_uncertainty_x.value),
                float(voxel_uncertainty_y.value),
                float(voxel_uncertainty_z.value)
            )
            seg_uncertainty = float(segmentation_uncertainty.value)
            
            # Measurement uncertainty
            uncertainty_result = measurement_uncertainty(
                current_volume, voxel_size, voxel_size_uncertainty, seg_uncertainty
            )
            progress_bar.value = 50
            
            # Compute metrics for confidence intervals
            metrics = compute_all_metrics(current_volume, voxel_size)
            progress_bar.value = 70
            
            # Confidence intervals (simplified - using uncertainty)
            conf_level = float(confidence_level.value)
            from scipy import stats
            z_score = stats.norm.ppf((1 + conf_level) / 2)
            
            volume_unc = uncertainty_result.get('volume_uncertainty_breakdown', {}).get('total', 0)
            volume_ci = (metrics.get('volume', 0) - z_score * volume_unc, 
                        metrics.get('volume', 0) + z_score * volume_unc)
            
            progress_bar.value = 90
            
            quality_results['uncertainty'] = {
                'measurement': uncertainty_result,
                'confidence_intervals': {
                    'volume': volume_ci,
                    'confidence_level': conf_level
                }
            }
            
            # Display results
            html = f"""
            <h4>📊 Uncertainty Analysis Results</h4>
            <p><b>Confidence Level:</b> {conf_level*100:.1f}%</p>
            <p><b>Volume Uncertainty:</b> {volume_unc:.4f} mm³ ({uncertainty_result.get('volume_uncertainty_breakdown', {}).get('relative', 0):.2f}%)</p>
            <p><b>Volume:</b> {metrics.get('volume', 0):.4f} ± {volume_unc:.4f} mm³</p>
            <p><b>Volume CI:</b> [{volume_ci[0]:.4f}, {volume_ci[1]:.4f}] mm³</p>
            <p><b>Surface Area Uncertainty:</b> {uncertainty_result.get('surface_area_uncertainty', 0):.4f} mm²</p>
            """
            uncertainty_results_display.value = html
            
            # Visualize
            with uncertainty_visualization:
                clear_output()
                fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                
                # Uncertainty breakdown
                breakdown = uncertainty_result.get('volume_uncertainty_breakdown', {})
                sources = ['Voxel Size', 'Segmentation', 'Total']
                values = [
                    breakdown.get('from_voxel_size', 0),
                    breakdown.get('from_segmentation', 0),
                    breakdown.get('total', 0)
                ]
                axes[0].bar(sources, values, alpha=0.7, color=['steelblue', 'coral', 'lightgreen'])
                axes[0].set_ylabel('Uncertainty (mm³)', fontsize=11)
                axes[0].set_title('Volume Uncertainty Breakdown', fontsize=12, fontweight='bold')
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Confidence interval
                axes[1].barh(['Volume'], [metrics.get('volume', 0)], alpha=0.7, color='steelblue')
                axes[1].errorbar([metrics.get('volume', 0)], [0], 
                               xerr=[[volume_unc], [volume_unc]], 
                               fmt='o', color='red', capsize=10, capthick=2, label=f'{conf_level*100:.0f}% CI')
                axes[1].set_xlabel('Volume (mm³)', fontsize=11)
                axes[1].set_title('Volume with Confidence Interval', fontsize=12, fontweight='bold')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3, axis='x')
                
                plt.tight_layout()
                plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Uncertainty analysis complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in uncertainty analysis: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def run_validation_callback(button):
        """Run validation analysis"""
        global analyzer, current_volume, quality_results
        
        if current_volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Running validation...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            
            # Compute our metrics
            our_metrics = compute_all_metrics(current_volume, voxel_size)
            progress_bar.value = 40
            
            method = validation_method.value
            tol = float(validation_tolerance.value) / 100.0  # Convert percentage to fraction
            
            if method == 'DragonFly Comparison':
                dragonfly_file_path = dragonfly_file.value.strip()
                if not dragonfly_file_path:
                    status_display.value = "<p style='color: red;'>Please provide DragonFly results file</p>"
                    return
                
                dragonfly_path = Path(dragonfly_file_path)
                if not dragonfly_path.exists():
                    dragonfly_path = project_root / 'data' / dragonfly_file_path
                    if not dragonfly_path.exists():
                        status_display.value = f"<p style='color: red;'>File not found: {dragonfly_file_path}</p>"
                        return
                
                progress_bar.value = 60
                
                # Compare with DragonFly
                comparison_result = compare_with_dragonfly(our_metrics, str(dragonfly_path), tolerance=tol)
                progress_bar.value = 90
                
                quality_results['validation'] = comparison_result
                
                # Display results
                html = f"""
                <h4>✅ DragonFly Comparison Results</h4>
                <p><b>Agreement:</b> {comparison_result.get('agreement_percentage', 0):.2f}%</p>
                <p><b>Metrics Compared:</b> {comparison_result.get('n_metrics_compared', 0)}</p>
                <p><b>Overall Agreement:</b> {'✅ Yes' if comparison_result.get('overall_agreement', False) else '❌ No'}</p>
                <h5>Metric Comparisons:</h5>
                <ul>
                """
                for metric, comp in comparison_result.get('comparisons', {}).items():
                    rel_diff = comp.get('relative_difference_percent', 0)
                    within = comp.get('within_tolerance', False)
                    html += f"<li><b>{metric}:</b> {rel_diff:.2f}% difference {'✅' if within else '❌'}</li>"
                html += "</ul>"
                validation_results_display.value = html
                
                # Visualize
                with validation_visualization:
                    clear_output()
                    fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                    
                    comparisons = comparison_result.get('comparisons', {})
                    metrics = list(comparisons.keys())
                    our_vals = [comparisons[m]['our_value'] for m in metrics]
                    dragonfly_vals = [comparisons[m]['dragonfly_value'] for m in metrics]
                    
                    axes.scatter(our_vals, dragonfly_vals, s=200, alpha=0.7)
                    # Add diagonal line
                    min_val = min(min(our_vals), min(dragonfly_vals))
                    max_val = max(max(our_vals), max(dragonfly_vals))
                    axes.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Agreement')
                    axes.set_xlabel('Our Results', fontsize=11)
                    axes.set_ylabel('DragonFly Results', fontsize=11)
                    axes.set_title('DragonFly Comparison', fontsize=12, fontweight='bold')
                    axes.legend()
                    axes.grid(True, alpha=0.3)
                    
                    plt.tight_layout()
                    plt.show()
                
            elif method == 'Ground Truth Validation':
                ground_truth_file_path = ground_truth_file.value.strip()
                if not ground_truth_file_path:
                    status_display.value = "<p style='color: red;'>Please provide ground truth file</p>"
                    return
                
                ground_truth_path = Path(ground_truth_file_path)
                if not ground_truth_path.exists():
                    ground_truth_path = project_root / 'data' / ground_truth_file_path
                    if not ground_truth_path.exists():
                        status_display.value = f"<p style='color: red;'>File not found: {ground_truth_file_path}</p>"
                        return
                
                progress_bar.value = 60
                
                # Load ground truth
                import json
                if ground_truth_path.suffix.lower() == '.json':
                    with open(ground_truth_path, 'r') as f:
                        ground_truth = json.load(f)
                else:
                    # Assume CSV
                    gt_df = pd.read_csv(ground_truth_path)
                    ground_truth = gt_df.to_dict('records')[0] if len(gt_df) > 0 else {}
                
                progress_bar.value = 80
                
                # Validate
                validation_result = validate_against_ground_truth(current_volume, ground_truth, voxel_size)
                progress_bar.value = 90
                
                quality_results['validation'] = validation_result
                
                # Display results
                html = f"""
                <h4>✅ Ground Truth Validation Results</h4>
                <p><b>Accuracy:</b> {validation_result.get('accuracy', {}).get('mean', 0):.2f}%</p>
                <p><b>Bias:</b> {validation_result.get('bias', {}).get('mean', 0):.4f}</p>
                <p><b>Precision:</b> {validation_result.get('precision', {}).get('mean', 0):.4f}</p>
                """
                validation_results_display.value = html
                
                # Visualize
                with validation_visualization:
                    clear_output()
                    fig, axes = plt.subplots(1, 1, figsize=(10, 6))
                    
                    # Plot accuracy metrics
                    metrics_list = list(validation_result.get('accuracy', {}).keys())
                    accuracy_vals = [validation_result['accuracy'][m] for m in metrics_list if m != 'mean']
                    axes.bar(metrics_list[:len(accuracy_vals)], accuracy_vals, alpha=0.7)
                    axes.set_ylabel('Accuracy (%)', fontsize=11)
                    axes.set_title('Validation Accuracy by Metric', fontsize=12, fontweight='bold')
                    axes.grid(True, alpha=0.3, axis='y')
                    
                    plt.tight_layout()
                    plt.show()
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Validation complete!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in validation: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    load_button.on_click(load_volume_callback)
    analyze_dimensional_button.on_click(analyze_dimensional_callback)
    analyze_uncertainty_button.on_click(analyze_uncertainty_callback)
    run_validation_button.on_click(run_validation_callback)
    
    # Enable/disable widgets based on validation method
    def update_validation_widgets(change):
        method = validation_method.value
        dragonfly_file.disabled = (method != 'DragonFly Comparison')
        ground_truth_file.disabled = (method != 'Ground Truth Validation')
    
    validation_method.observe(update_validation_widgets, names='value')
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


## 5. Display Interactive Dashboard


In [5]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Segmented Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        HBox([load_button, volume_info_display])
    ])
    
    # Create dimensional accuracy panel
    dimensional_panel = widgets.VBox([
        widgets.HTML("<h3>📏 Dimensional Accuracy</h3>"),
        widgets.HTML("<b>Design Dimensions:</b>"),
        HBox([design_dim_x, design_dim_y, design_dim_z]),
        tolerance,
        analyze_dimensional_button,
        dimensional_results_display,
        dimensional_visualization
    ])
    
    # Create uncertainty panel
    uncertainty_panel = widgets.VBox([
        widgets.HTML("<h3>📊 Uncertainty Analysis</h3>"),
        widgets.HTML("<b>Voxel Size Uncertainty:</b>"),
        HBox([voxel_uncertainty_x, voxel_uncertainty_y, voxel_uncertainty_z]),
        segmentation_uncertainty,
        confidence_level,
        analyze_uncertainty_button,
        uncertainty_results_display,
        uncertainty_visualization
    ])
    
    # Create validation panel
    validation_panel = widgets.VBox([
        widgets.HTML("<h3>✅ Validation</h3>"),
        validation_method,
        dragonfly_file,
        ground_truth_file,
        validation_tolerance,
        run_validation_button,
        validation_results_display,
        validation_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        dimensional_panel,
        uncertainty_panel,
        validation_panel
    ])
    analysis_tabs.set_title(0, '📏 Dimensional')
    analysis_tabs.set_title(1, '📊 Uncertainty')
    analysis_tabs.set_title(2, '✅ Validation')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>✅ Quality Control and Validation</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start quality control and validation analysis.")
    print("\n💡 Tips:")
    print("   1. Load a segmented volume")
    print("   2. Set design dimensions and tolerance for dimensional accuracy")
    print("   3. Configure uncertainty parameters")
    print("   4. Validate against DragonFly or ground truth")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start quality control and validation analysis.

💡 Tips:
   1. Load a segmented volume
   2. Set design dimensions and tolerance for dimensional accuracy
   3. Configure uncertainty parameters
   4. Validate against DragonFly or ground truth


In [6]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Segmented Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        HBox([load_button, volume_info_display])
    ])
    
    # Create dimensional accuracy panel
    dimensional_panel = widgets.VBox([
        widgets.HTML("<h3>📏 Dimensional Accuracy</h3>"),
        widgets.HTML("<b>Design Dimensions:</b>"),
        HBox([design_dim_x, design_dim_y, design_dim_z]),
        tolerance,
        analyze_dimensional_button,
        dimensional_results_display,
        dimensional_visualization
    ])
    
    # Create uncertainty panel
    uncertainty_panel = widgets.VBox([
        widgets.HTML("<h3>📊 Uncertainty Analysis</h3>"),
        widgets.HTML("<b>Voxel Size Uncertainty:</b>"),
        HBox([voxel_uncertainty_x, voxel_uncertainty_y, voxel_uncertainty_z]),
        segmentation_uncertainty,
        confidence_level,
        analyze_uncertainty_button,
        uncertainty_results_display,
        uncertainty_visualization
    ])
    
    # Create validation panel
    validation_panel = widgets.VBox([
        widgets.HTML("<h3>✅ Validation</h3>"),
        validation_method,
        dragonfly_file,
        ground_truth_file,
        validation_tolerance,
        run_validation_button,
        validation_results_display,
        validation_visualization
    ])
    
    # Create tabs for organized display
    analysis_tabs = Tab(children=[
        dimensional_panel,
        uncertainty_panel,
        validation_panel
    ])
    analysis_tabs.set_title(0, '📏 Dimensional')
    analysis_tabs.set_title(1, '📊 Uncertainty')
    analysis_tabs.set_title(2, '✅ Validation')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>✅ Quality Control and Validation</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        widgets.HTML("<h2>📊 Analysis</h2>"),
        analysis_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Start quality control and validation analysis.")
    print("\n💡 Tips:")
    print("   1. Load a segmented volume")
    print("   2. Set design dimensions and tolerance for dimensional accuracy")
    print("   3. Configure uncertainty parameters")
    print("   4. Validate against DragonFly or ground truth")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Start quality control and validation analysis.

💡 Tips:
   1. Load a segmented volume
   2. Set design dimensions and tolerance for dimensional accuracy
   3. Configure uncertainty parameters
   4. Validate against DragonFly or ground truth


## 6. Summary

### What We Learned

1. **Dimensional Accuracy**:
   - Geometric deviation calculation
   - Tolerance compliance analysis
   - Design vs. actual comparison
   - RMS and max deviation metrics

2. **Uncertainty Analysis**:
   - Measurement uncertainty from voxel size and segmentation
   - Uncertainty breakdown by source
   - Confidence interval calculation
   - Relative uncertainty percentages

3. **Validation**:
   - DragonFly software comparison
   - Ground truth validation
   - Accuracy, bias, and precision metrics
   - Agreement percentage calculation

### Key Insights

- **Dimensional accuracy** helps assess manufacturing quality
- **Uncertainty quantification** provides confidence in measurements
- **Validation** ensures analysis correctness
- **Quality control** metrics enable process improvement

### Next Steps

- **Notebook 08**: Complete Analysis Pipeline
  - End-to-end workflow
  - Comprehensive reporting
  - Reproducibility package

### Resources

- [Framework Documentation](../docs/README.md)
- [Quality Modules](../docs/modules.md#quality-modules)
- [Dimensional Accuracy](../docs/modules.md#qualitydimensional_accuracy)
- [Uncertainty Analysis](../docs/modules.md#qualityuncertainty_analysis)
- [Validation](../docs/modules.md#qualityvalidation)
